In [ ]:
tbl = '/mnt/c/mystuff/rms/equipasaude/areainf_cs_2021_v3.xlsx'

cscol = 'ID'

fregcol = 'fregid'

facilities     = '/mnt/c/mystuff/rms/equipasaude/csaude2021.shp'
idfac = 'ID'

incidents      = '/mnt/c/mystuff/rms/admin_data/rms_bgripnt.shp'
fregpk = 'idfreg'

outfolder = '/mnt/c/mystuff/rms/dist_cs'

In [ ]:
import os
from glass.rd import tbl_to_obj
from glass.rd.shp import shp_to_obj
from glass.pys.oss import mkdir
from glass.wt.shp import df_to_shp

In [ ]:
df = tbl_to_obj(tbl)

In [ ]:
df[df.fregid.isna()]

In [ ]:
df[fregcol] = df[fregcol].str.replace('\t', '')

In [ ]:
fregs = df[fregcol].unique()

In [ ]:
fregs_cs = {}
fregs_ia = {}

for freg in fregs:
    fcs = df[df[fregcol] == freg]
    
    fregs_cs[freg] = list(fcs[cscol].unique())
    
    fregs_cs[freg].sort()
    
    fregs_ia[freg] = 0

In [ ]:
# Get Influence Areas

iareas = {}

iaid = 1
for i in range(len(fregs)):
    if fregs_ia[fregs[i]]:
        continue
    
    iareas[iaid] = [fregs[i]]
    
    fregs_ia[fregs[i]] = 1
    
    for e in range(i+1, len(fregs)):
        if fregs_ia[fregs[e]]:
            continue
        
        if fregs_cs[fregs[i]] == fregs_cs[fregs[e]]:
            iareas[iaid].append(fregs[e])
            
            fregs_ia[fregs[e]] = 1
            
            continue
    
    iaid += 1

In [ ]:
iareas[361]

In [ ]:
# For each iarea, get incidents and facilities

# Obter as facilities
fdf = shp_to_obj(facilities)

# Get incidents
idf = shp_to_obj(incidents)

In [ ]:
for ia in iareas:
    # Select incidents
    s_idf = idf[idf[fregpk].isin(iareas[ia])]
    
    # Select facilities
    s_fdf = fdf[fdf[idfac].isin(fregs_cs[iareas[ia][0]])]
    
    if not s_idf.shape[0] or not s_fdf.shape[0]:
        print('Start Warning')
        print(ia)
        print(iareas[ia])
        print(fregs_cs[iareas[ia][0]])
        print('===============\n===========\n')
    
    # Make dir
    iafolder = mkdir(os.path.join(outfolder, f'ia_{str(ia)}'))
    
    # Save shapefiles
    df_to_shp(s_idf, os.path.join(iafolder, 'incidents.shp'))
    df_to_shp(s_fdf, os.path.join(iafolder, 'facilities.shp'))